In [60]:
%matplotlib inline
import matplotlib.pyplot as plt

import mne
mne.set_log_level('critical')

import numpy as np
import scipy
import pandas as pd
import numpy as np
import copy
from os import listdir

from mne.preprocessing import create_eog_epochs, find_outliers
from mne.io import set_bipolar_reference as bipolar
from autoreject import LocalAutoRejectCV

montage = '~/conincon_data/m.elp' # load montage file
event_ids = {"con/hc":201, "con/lc":200, "inc/lc":210, "inc/hc":211} # event ids and associated triggers

def get_name(inst):
    if isinstance(inst, str):
        i_ = inst
    else:
        i_ = inst.info["filename"]
    return i_.split("/")[-1].split("-")[0]

In [89]:
loc_recs="~/conincon_data/" # location of recordings (vhdr)
loc_files = loc_recs+'files/' # location to save the icas & the autorejections
template = loc_files+"{name}-raw.fif" # template to save individual raw filea

In [90]:
files = [loc_recs + f for f in listdir(loc_recs) if "sentence.vhdr" in f] # list of directory for individual subject data


In [91]:
def preproc(m, plot=True):  
    '''
    Preprocess data, m corresponds to file directory (i.e. /location_of_rawfiles/AJA31_sentence.vhdr)
    '''
    name = m.split("/")[-1].split("_")[0] # get name of participant (pseudonym)
    raw = mne.io.read_raw_brainvision(m, montage=montage, preload=True, eog=[]) # read subject data data
    events = mne.find_events(raw, shortest_event=0, min_duration=0, consecutive=True) #find events
    raw, events = raw.resample(100, npad='auto', n_jobs=12, events=events) # downsample to 100 hz to facilitate MVPA

    if "M1" in list(raw.ch_names): 
        # for data that was not online referenced to mastoids, re-reference to linked mastoids
        m = mne.channels.montage.read_montage(montage)
        mne.channels.montage._set_montage(raw.info, m)
        mne.io.set_eeg_reference(raw, ref_channels=["M1", "M2"], copy=False) # re-reference to linked mastoids
        raw.drop_channels(["M1", "M2"]) #drop the mastoid channels
    params = dict(name=name)    
    raw.save(template.format(**params), overwrite=True)


In [ ]:
%%capture
missing = []
for ii, m in enumerate(files):
    try:
        preproc(m) # preprocess each file
    except Exception as e:
        print(e)
        missing.append([m, e]) 
        #creates a list of problematic files if there are any where the preprocessing didn't work

In [ ]:
# creates a list of participant names based on the saved raw files from earlier
names = list({fname.split("-")[0]  
              for fname in listdir(loc_files) 
                if "-raw.fif" in fname
             })



In [ ]:
def get_epoch(name, times=dict(tmin=-0.3, tmax=1.3)):
    
    '''
    Loads the preprocessed raw files and creates an epoch used for ICA correction.
    name: paricipant
    times: start and end of epoch (in this case actually uses -.8 and.8)
    '''
    # create a function to load epochs for ICA correction
    # name is the participant
    params = dict(name=name)
    
    with mne.io.read_raw_fif(template.format(**params), preload=True) as raw:
        picks = mne.pick_types(raw.info, eeg=True, stim=False)
        events = mne.find_events(raw, min_duration=0, shortest_event=0)

        return mne.Epochs(raw, events, event_id=event_ids,preload=True,
                          baseline=(None, None), picks=picks, reject=dict(), **times)


def calculate_ica_and_ar(name):
    
    '''
    Get participants ICAs and autorejection/repair.
    '''
    
    #ICA corr
    ica_times = dict(tmin=-.8, tmax=.8)
    epochs = get_epoch(name, ica_times)
    ar = LocalAutoRejectCV()
    epochs = ar.fit_transform(epochs)  # autoreject for channel interpolation and trial rejection
    ica = mne.preprocessing.ica.ICA(n_components=30).fit(epochs)  # ICA for EOG
    return {"ica":ica, "ar":ar}

In [ ]:
%%capture

missing = []  

preprocs=dict() # preprocessing for ICAs and Autorejects
for ii, name in enumerate(names):
    print(ii, name)
    if name not in preprocs and name not in missing:
        preprocs[name] = calculate_ica_and_ar(name)

In [ ]:
# using arrays as templates for corrmap
# these are hand-picked example blinks and saccades from a handful of ICAs for these subjects
# you can probably use these every time you have the same montage (channel configuratio)

blink1 = [-0.13222267, -0.58994499, -0.29355172, -0.18577413, -0.13666416, -0.12118263, -0.06680473, -0.09495492, -0.04173736, -0.00795334,         0.01092489,  0.03284364,  0.01576397,  0.03195119,  0.06791669,         0.1314736 ,  0.06504867,  0.06324997,  0.07419753,  0.11725419,         0.11774556,  0.11719014,  0.10702447,  0.13149544,  0.16260529,        -0.0111242 ,  0.16680825,  0.18126139,  0.16469281, -0.31963016,        -0.56935977, -0.37826227, -0.31406649, -0.3957065 , -0.27899182,        -0.15266724, -0.09900878, -0.15983442,  0.51298219, -0.05923843,        -0.06955318,  0.00525162,  0.08179958,  0.61220865,  0.00662278,         0.01853569,  0.04376732,  0.08917384,  0.08657535,  0.06865508,         0.05902028,  0.09452821,  0.14441957,  0.11683305,  0.11525051,         0.12160206,  0.1354917 ,  0.15915092,  0.14567324,  0.16016644,         0.14163399,  0.16058371]  
blink3 = [-0.19851515, -0.20510679, -0.87775481, -0.80763199, -0.79732761, -0.92204281, -1.00216934, -0.54353195, -0.55290501, -0.57912892,        -0.29458718, -0.65296413, -0.40697541, -0.39600718, -0.46630531,        -0.34020572, -0.28454487, -0.30169799, -0.32000772, -0.3086994 ,        -0.15609367, -0.22312092, -0.20498235, -0.20828604, -0.16831942,        -0.54323034, -0.07863662, -0.08025507, -0.10480094, -1.27000752,        -1.58455468, -1.20712029, -1.36163807, -1.8824352 , -0.87890347,        -0.74799237, -0.81150975, -1.072836  ,  1.2502202 , -0.47498204,        -0.55165773, -0.6401265 , -0.57039403,  1.24870392, -0.37407656,        -0.40863226, -0.4354275 , -0.44090026, -0.21091885, -0.29651021,        -0.29088014, -0.33438976, -0.25501006, -0.19793369, -0.21956021,        -0.21963283, -0.19074065, -0.11153281, -0.14093191, -0.13524869,        -0.13716129, -0.10653287]  
blink2 = [ 0.18320148,  0.21334169, -0.43004667, -0.37972791, -0.28788174, -0.28607842, -0.23319324, -0.13627579, -0.10157262, -0.0806379 ,        -0.01231962,  0.09392891, -0.00475743,  0.02379593,  0.06805464,         0.18951154,  0.12414157,  0.07417806,  0.08881405,  0.16316773,         0.24697288,  0.138413  ,  0.13662858,  0.18058314,  0.2538965 ,        -0.0812754 ,  0.27533355,  0.27734751,  0.29125456, -0.57770343,        -0.10977603, -0.68888908, -0.60267863, -1.08824129, -0.41723556,        -0.31589346, -0.26583006, -0.29084337,  0.7897253 , -0.09310728,        -0.13099108, -0.03408091,  0.08687967,  1.1296567 ,  0.04035525,         0.00479207,  0.02958008,  0.10263463,  0.20729952,  0.0766577 ,         0.08664292,  0.1277247 ,  0.24572245,  0.18225315,  0.12313163,         0.14275728,  0.21756085,  0.24916469,  0.19611483,  0.19726039,         0.23660754,  0.29041034]  
blink4 = [ 0.19971999,  0.08511882, -0.14917381,  0.39965808,  0.17688549, 0.48875617, -0.62970161,  0.10776543,  0.10859065,  0.12875672,         0.46977464, -0.333145  ,  0.0436886 ,  0.05356912,  0.08545244,        -0.25123941, -0.07081378,  0.01926105,  0.02398057,  0.02228598,        -0.08378208, -0.0473971 , -0.0131947 , -0.07033687, -0.19247909,         0.08196583, -0.06451834, -0.06403569, -0.0841763 ,  0.26178197,         0.11796369,  0.48623019,  0.52026631, -0.25601062,  0.50544332,         0.22342059,  0.24573447,  0.52988724, -0.19839615, -0.11567741,         0.14030183,  0.27243695, -0.31372044, -0.17399436,  0.06986442,         0.05628076,  0.05978156,  0.04816653, -0.18265278,  0.00355977,         0.02963612,  0.00867089, -0.23771215, -0.04818989, -0.01774703,        -0.02697667, -0.10250253, -0.0673711 , -0.05704644, -0.05995935,        -0.08843798, -0.1073788 ]  
blink5 = [ -4.95394865e-03,  -6.16519433e-01,  -1.13142214e-01, -1.56382318e-01,  -1.11849827e-01,  -1.50726955e-01,         -3.66862903e-02,  -5.03920993e-02,  -5.08901168e-02,         -4.83993096e-02,  -6.52900792e-02,   8.01093167e-02,         -4.74331734e-04,  -5.62898444e-03,  -1.90525266e-03,          1.02989871e-01,   4.82753890e-02,   1.87195248e-02,          2.27863020e-02,   6.61828206e-02,   9.89180170e-02,          6.21336916e-02,   5.79431910e-02,   7.05616333e-02,          1.08969265e-01,  -4.61897098e-02,   1.08992044e-01,          1.21229938e-01,   1.30666712e-01,  -2.32735494e-01,          1.54037457e-02,  -2.72742101e-01,  -2.28492089e-01,         -7.39669092e-03,  -2.21026382e-01,  -1.10173489e-01,         -1.16178409e-01,  -1.44192330e-01,   5.62343041e-01,          6.94571235e-05,  -6.08154365e-02,  -5.91259552e-02,          4.43264165e-02,   7.99083522e-01,   4.05207218e-03,         -9.83000665e-03,  -7.04831704e-03,   2.70843180e-02,          8.49984035e-02,   2.53502618e-02,   2.80338644e-02, 3.73832768e-02,   1.23185197e-01,   6.38429280e-02, 4.69251903e-02,   5.20781429e-02,   9.55234146e-02, 9.49961458e-02,   8.32077983e-02,   8.74990325e-02, 9.81405522e-02,   1.26087163e-01]
blink6 = [-0.6245844 ,  0.13131128, -0.30223345, -0.35631716, -0.2381301 , -0.45679521, -0.42361314, -0.02517527, -0.0289743 , -0.02960042, -0.03271241,  0.19500501,  0.09899577,  0.10318531,  0.10191389, 0.19482378,  0.20183988,  0.16424488,  0.17315674,  0.23255274, 0.27761383,  0.22346029,  0.22735048,  0.24242078,  0.2872133 , -0.0210072 ,  0.30079017,  0.29011091,  0.28238481, -0.53890509, -1.05845617, -0.71964004, -0.54399788,  0.09573145, -0.36715467, -0.21883955, -0.22448845, -0.44527456, -0.4121691 ,  0.03457561, -0.03691665, -0.05191654,  0.01308568,  1.01953702,  0.12184037, 0.0963061 ,  0.0971582 ,  0.13554435,  0.26126075,  0.17008321, 0.17903516,  0.18502583,  0.26831014,  0.25191231,  0.21711559, 0.23842699,  0.25248564,  0.30681209,  0.25938976,  0.26975932, 0.2685889 ,  0.30656449]
blink7 = [ -1.33149772e+00,  -1.33059624e+00,  -3.63457539e-01,  -4.05428119e-01,  -2.53409687e-01,  -4.90342090e-01,  -3.89531351e-01,  -4.08718083e-02,  -4.05093474e-02,   1.26098276e-03,   3.75392712e-02,   2.33896862e-01,   9.79767525e-02,   1.11321022e-01,   1.35754749e-01,   2.89357368e-01,   2.25056844e-01,   1.74457755e-01,   1.90029822e-01,   2.58845047e-01,   3.19529656e-01,   2.36406546e-01,   2.35645347e-01,   2.73472429e-01,   3.39745430e-01,  -7.61338157e-03,   3.20040838e-01,   3.12282919e-01,   3.20151887e-01,  -6.13210081e-01,  -1.07101984e+00,  -8.74439694e-01,  -5.93583365e-01,  -1.17123292e+00,  -4.51854134e-01,  -2.42435207e-01,  -2.12397603e-01,  -4.71573293e-01,  -1.43259993e+00,   4.91605082e-02,  -6.49900329e-02,  -4.40987858e-02,   1.07375589e-01,  -1.50847033e+00,   1.24335123e-01,   9.20374894e-02,   1.17715606e-01,   2.03094995e-01,   2.94199062e-01,   1.69521655e-01,   1.79597262e-01,   2.12572639e-01,   3.35633058e-01,   2.66163038e-01,   2.23795327e-01,   2.43333911e-01,   2.86449372e-01,   2.96377321e-01,   2.81758143e-01,   2.79017373e-01,   2.78172515e-01,   3.18261549e-01]
saccade1 = [-0.18662803,  0.4292021 , -0.70139228, -0.24620428, -0.0021902 ,         0.35724515,  0.88948001, -0.43274324, -0.12945539,  0.0862682 ,         0.54999431, -0.36414012, -0.22234465, -0.05311555,  0.19288893,         0.44507754, -0.23890733, -0.12596169, -0.00844768,  0.16034008,        -0.20494805, -0.1707112 , -0.10989835, -0.03049865,  0.04647347,        -0.02501222, -0.14564911, -0.1572492 , -0.11508588,  0.04700906,        -0.59426749, -0.18116428,  0.35225356,  0.99506519, -0.47286443,        -0.1228715 ,  0.11758121,  0.639061  , -0.29387522, -0.51377554,        -0.26232986,  0.29536009,  0.64040682,  1.21988711, -0.31524576,        -0.12511567,  0.05436995,  0.34815278, -0.26671833, -0.18659605,        -0.06663742,  0.05628291,  0.20190854, -0.20840197, -0.15005357,        -0.0619991 ,  0.00898678, -0.19839085, -0.17624553, -0.13648224,        -0.09387813, -0.06826599] 

all_artefacts = [saccade1, blink1, blink2, blink3, blink4, blink5, blink6, blink7]

In [ ]:
icas = dict() # create a dict for ICAs
for name in names:
    icas[name]=preprocs[name]["ica"]

autorejs = dict() # create a dict for autorejs
for name in names:
    autorejs[name]=preprocs[name]["ar"]
    

In [ ]:
for name in names:
    for artefact in all_artefacts:  # mark artefact ICs for each subj
        mne.preprocessing.ica.corrmap(list(icas.values()), np.asarray(artefact),
                                      label="eog", threshold=.8,
                                      plot=False, show=False);


In [ ]:
#SAVE ICA and Autoreject
import pickle
with open(loc_files +'ar.pckl', 'wb') as f:
        pickle.dump(autorejs, f)

for name in icas:
    icas[name].save(loc_files+'{}-ica.fif'.format(name))

    